# Module 1 - Programming Assignment

## General Directions

1. You must follow the Programming Requirements outlined on Canvas.
2. The Notebook should be cleanly and fully executed before submission.
3. You should change the name of this file to be your JHED id. For example, `jsmith299.ipynb` although Canvas will change it to something else... :/

<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        You should always read the entire assignment before beginning your work, so that you know in advance what the requested output will be and can plan your implementation accordingly.
    </p>
</div>

# State Space Search with A* Search

You are going to implement the A\* Search algorithm for navigation problems.


## Motivation

Search is often used for path-finding in video games. Although the characters in a video game often move in continuous spaces,
it is trivial to layout a "waypoint" system as a kind of navigation grid over the continuous space. Then if the character needs
to get from Point A to Point B, it does a line of sight (LOS) scan to find the nearest waypoint (let's call it Waypoint A) and
finds the nearest, LOS waypoint to Point B (let's call it Waypoint B). The agent then does a A* search for Waypoint B from Waypoint A to find the shortest path. The entire path is thus Point A to Waypoint A to Waypoint B to Point B.

We're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they
specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
🌾       plains     1
🌲       forest     3
⛰       hills      5
🐊       swamp      7
🌋       mountains  impassible
```

We can think of the raw format of the map as being something like:

```
🌾🌾🌾🌾🌲🌾🌾
🌾🌾🌾🌲🌲🌲🌾
🌾⛰⛰⛰🌾🌾🌾
🌾🌾⛰⛰🌾🌾🌾
🌾🌾⛰🌾🌾🌲🌲
🌾🌾🌾🌾🌲🌲🌲
🌾🌾🌾🌾🌾🌾🌾
```

## The World

Given a map like the one above, we can easily represent each row as a `List` and the entire map as `List of Lists`:

In [1]:
full_world = [
['🌾', '🌾', '🌾', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌋', '🌋', '🌋', '🌋', '🌋', '🌋', '🌋', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '🌋', '⛰', '⛰', '⛰', '🌋', '🌋', '⛰', '⛰'],
['🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🐊', '🐊', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '⛰', '🌾'],
['🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '🌲', '🌲', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '🌋', '⛰', '🌾'],
['🌾', '⛰', '⛰', '⛰', '🌋', '🌋', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '⛰', '🌾', '🌾'],
['🌾', '⛰', '⛰', '🌋', '🌋', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '🌋', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌾', '🌾', '🌾'],
['🌾', '🌾', '⛰', '⛰', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '🌋', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾'],
['🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '🌾', '🐊', '🐊', '🌾', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '⛰', '⛰', '🌋', '🌋', '🌋', '🌋', '🌾', '🌾', '🌾', '🐊', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '⛰', '⛰', '🌋', '🌋', '🌋', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '🌋', '⛰', '🌾', '🌾', '🌾'],
['🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '🌋', '🌋', '⛰', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🌾', '🌾', '⛰', '🌋', '🌋', '⛰', '🌾', '🌾', '🌾'],
['🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '⛰', '🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '⛰', '🌋', '⛰', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '🌲', '🌲', '⛰', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🌋', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '⛰', '⛰', '⛰', '⛰', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '⛰', '🌋', '⛰', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌋', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '🌋', '⛰', '⛰', '🌾', '🐊', '🌾', '⛰', '🌋', '🌋', '⛰', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '🌋', '🌾', '🌾', '🌋', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌋', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🌋', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '⛰', '⛰', '⛰', '⛰', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌋', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '🌋', '🌋', '🌋', '🌲', '🌲', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '🌋', '🌋', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '⛰', '⛰', '🌾', '🌾', '⛰', '⛰', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '⛰', '⛰', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊'],
['⛰', '🌋', '⛰', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌋', '🌋', '🌋', '⛰', '⛰', '🌋', '🌋', '🌾', '🌋', '🌋', '⛰', '⛰', '🐊', '🐊', '🐊', '🐊'],
['⛰', '🌋', '🌋', '🌋', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '🌋', '🌋', '⛰', '⛰', '⛰', '⛰', '🌋', '🌋', '🌋', '🐊', '🐊', '🐊', '🐊'],
['⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾']
]

In [2]:
for line in full_world:
    print("".join(line))

🌾🌾🌾🌾🌾🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾
🌾🌾🌾🌾🌾🌾🌾🌲🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🌋🌋🌋🌋🌋🌋🌋🌾🌾
🌾🌾🌾🌾🌋🌋🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌋🌋🌋⛰⛰⛰🌋🌋⛰⛰
🌾🌾🌾🌾⛰🌋🌋🌋🌲🌲🌲🌲🐊🐊🌲🌲🌲🌲🌲🌾🌾⛰⛰🌋🌋⛰🌾
🌾🌾🌾⛰⛰🌋🌋🌲🌲🌾🌾🐊🐊🐊🐊🌲🌲🌲🌾🌾🌾⛰🌋🌋🌋⛰🌾
🌾⛰⛰⛰🌋🌋⛰⛰🌾🌾🌾🌾🐊🐊🐊🐊🐊🌾🌾🌾🌾🌾⛰🌋⛰🌾🌾
🌾⛰⛰🌋🌋⛰⛰🌾🌾🌾🌾⛰🌋🌋🌋🐊🐊🐊🌾🌾🌾🌾🌾⛰🌾🌾🌾
🌾🌾⛰⛰⛰⛰⛰🌾🌾🌾🌾🌾🌾⛰🌋🌋🌋🐊🐊🐊🌾🌾⛰⛰⛰🌾🌾
🌾🌾🌾⛰⛰⛰🌾🌾🌾🌾🌾🌾⛰⛰🌋🌋🌾🐊🐊🌾🌾⛰⛰⛰🌾🌾🌾
🌾🌾🌾🐊🐊🐊🌾🌾⛰⛰⛰🌋🌋🌋🌋🌾🌾🌾🐊🌾⛰⛰⛰🌾🌾🌾🌾
🌾🌾🐊🐊🐊🐊🐊🌾⛰⛰🌋🌋🌋⛰🌾🌾🌾🌾🌾⛰🌋🌋🌋⛰🌾🌾🌾
🌾🐊🐊🐊🐊🐊🌾🌾⛰🌋🌋⛰🌾🌾🌾🌾🐊🐊🌾🌾⛰🌋🌋⛰🌾🌾🌾
🐊🐊🐊🐊🐊🌾🌾⛰⛰🌋🌋⛰🌾🐊🐊🐊🐊🌾🌾🌾⛰🌋⛰🌾🌾🌾🌾
🌾🐊🐊🐊🐊🌾🌾⛰🌲🌲⛰🌾🌾🌾🌾🐊🐊🐊🐊🌾🌾⛰🌾🌾🌾🌾🌾
🌾🌾🌾🌾🌋🌾🌾🌲🌲🌲🌲⛰⛰⛰⛰🌾🐊🐊🐊🌾🌾⛰🌋⛰🌾🌾🌾
🌾🌾🌾🌋🌋🌋🌲🌲🌲🌲🌲🌲🌋🌋🌋⛰⛰🌾🐊🌾⛰🌋🌋⛰🌾🌾🌾
🌾🌾🌋🌋🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌋🌋🌋🌾🌾🌋🌋🌋🌾🌾🌾🌾🌾
🌾🌾🌾🌋🌋🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌋🌋🌋🌋🌾🌾🌾🌾🌾🌾🌾
🌾🌾🌾🌋🌋🌋🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🌾⛰⛰🌾🌾🌾🌾🌾🌾🌾🌾
🌾🌾🌾🌾🌋🌋🌋🌲🌲🌲🌲🌲🌲🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🐊🐊🐊🐊
🌾🌾⛰⛰⛰⛰🌋🌋🌲🌲🌲🌲🌲🌾🌋🌾🌾🌾🌾🌾🐊🐊🐊🐊🐊🐊🐊
🌾🌾🌾🌾⛰⛰⛰🌋🌋🌋🌲🌲🌋🌋🌾🌾🌾🌾🌾🌾🐊🐊🐊🐊🐊🐊🐊
🌾🌾🌾🌾🌾🌾⛰⛰⛰🌋🌋🌋🌋🌾🌾🌾🌾⛰⛰🌾🌾🐊🐊🐊🐊🐊🐊
🌾⛰⛰🌾🌾⛰⛰⛰⛰⛰🌾🌾🌾🌾🌾⛰⛰🌋🌋⛰⛰🌾🐊🐊🐊🐊🐊
⛰🌋⛰⛰⛰⛰🌾🌾🌾🌾🌾🌋🌋🌋⛰⛰🌋🌋🌾🌋🌋⛰⛰🐊🐊🐊🐊
⛰🌋🌋🌋⛰🌾🌾🌾🌾🌾⛰⛰🌋🌋🌋🌋⛰⛰⛰⛰🌋🌋🌋🐊🐊🐊🐊
⛰⛰🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾⛰⛰⛰⛰⛰🌾🌾🌾🌾⛰⛰⛰🌾🌾🌾


## Warning

One implication of this representation is that (x, y) is world[ y][ x] so that (3, 2) is world[ 2][ 3] and world[ 7][ 9] is (9, 7). Yes, there are many ways to do this. I picked this representation because when you look at it, it *looks* like a regular x, y cartesian grid and it's easy to print out.

It is often easier to begin your programming by operating on test input that has an obvious solution. If we had a small 7x7 world with the following characteristics:

In [3]:
small_world = [
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾']
]



**what do you expect the policy would be?** Think about it for a bit. This will help you with your programming and debugging.

## States and State Representation

The canonical pieces of a State Space Search problem are the States, Actions, Transitions and Costs. 

We'll start with the state representation. For the navigation problem, a state is the current position of the agent, `(x,y)`. The entire set of possible states is implicitly represented by the world map.

## Actions and Transitions

Next we need to specify the actions. In general, there are a number of different possible action sets in such a world. The agent might be constrained to move north/south/east/west or diagonal moves might be permitted as well (or really anything). When combined with the set of States, the *permissible* actions forms the Transition set.

Rather than enumerate the Transition set directly, for this problem it's easier to calculate the available actions and transitions on the fly. This can be done by specifying a *movement model* as offsets to the current state and then checking to see which of the potential successor states are actually permitted. This can be done in the successor function mentioned in the pseudocode.

One such example of a movement model is shown below.

In [4]:
MOVES = [(0,-1), (1,0), (0,1), (-1,0)]

## Costs

We can encode the costs described above in a `Dict`:

In [5]:
COSTS = { '🌾': 1, '🌲': 3, '⛰': 5, '🐊': 7}

## Specification

You will implement a function called `a_star_search` that takes the parameters and returns the value as specified below. The return value is going to look like this:

`[(0,1), (0,1), (0,1), (1,0), (1,0), (1,0), (1,0), (1,0), (1,0), (0,1), (0,1), (0,1)]`

You should also implement a function called `pretty_print_path`. 
The `pretty_print_path` function prints an ASCII representation of the path generated by the `a_star_search` on top of the terrain map. 
For example, for the test world, it would print this:

```
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏩⏩⏩⏩⏩⏩⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲🎁
```

using ⏩,⏪,⏫ ⏬ to represent actions and `🎁` to represent the goal. (Note the format of the output...there are no spaces, commas, or extraneous characters). You are printing the path over the terrain.
This is an impure function (because it has side effects, the printing, before returning anything).

Note that in Python:
```
> a = ["*", "-", "*"]
> "".join(a)
*-*
```
* Do not print raw data structures
* Do not insert unneeded/unrequested spaces
* Use the provided emojis!

### Additional comments

As Python is an interpreted language, you're going to need to insert all of your functions *before* the actual `a_star_search` function implementation. 
Do not make unwarranted assumptions (for example, do not assume that the start is always (0, 0).
Do not refer to global variables, pass them as parameters (functional programming).

Simple and correct is better than inefficient and incorrect, or worse, incomplete.
For example, you can use a simple List, with some helper functions, as a Stack or a Queue or a Priority Queue.
Avoid the Python implementations of HeapQ, PriorityQueue implementation unless you are very sure about what you're doing as they require *immutable* keys.

In [6]:
from typing import List, Tuple, Dict, Callable
from copy import deepcopy

*add as many markdown and code cells here as you need for helper functions. We have added `heuristic` for you*

<a id="priority_sort"></a>
## priority_sort
`priority_sort` takes a dictionary of states (`Dict[Tuple[int, int], Tuple[int, int, int]]`). It sorts the list in reverse order by the third element in the value tuple - the cost of reaching the key's position. The queue is reverse-sorted so that the lowest priority is popped off the back of the queue in the `a_star_search` function. **Used by:** [a_star_search](#a_star_search)

* **queue**: the dictionary with tile coordinates as the keys and moves/cost as the values.

**returns** Dict[Tuple[int, int], Tuple[int, int, int]]:: the queue that was passed as a parameter in priority-sorted order (highest to lowest priority)

In [7]:
def priority_sort(queue: Dict[Tuple[int, int], Tuple[int, int, int]]) -> Dict[Tuple[int, int], Tuple[int, int, int]]:
    return dict(sorted(queue.items(), reverse=True, key=lambda priority: priority[1][2]))

In [8]:
# assertions/unit tests
q = {(1, 3): (1, 1, 3), (1, 2): (1, 1, 2), (1, 1): (1, 1, 1)}
actual_sorted = priority_sort(q)
assert str(actual_sorted) == "{(1, 3): (1, 1, 3), (1, 2): (1, 1, 2), (1, 1): (1, 1, 1)}"

q = {(1, -3): (1, 1, -3), (1, -2): (1, 1, -2), (1, -1): (1, 1, -1)}
actual_sorted = priority_sort(q)
assert str(actual_sorted) == "{(1, -1): (1, 1, -1), (1, -2): (1, 1, -2), (1, -3): (1, 1, -3)}"

q = {}
actual_sorted = priority_sort(q)
assert str(actual_sorted) == "{}"

<a id="path"></a>
## path

`path` takes a start node, destination node and an explored list, and returns the path from the destination node to the start node. The explored list is a dictionary that maps a node to the move taken to reach that node. Every element is added to a stack and the stack is reversed at the end to return an in-order list of moves to take to get from the destination to the start node. **Used by:** [a_star_search](#a_star_search)

* **start**: the node at which we begin backtracking our path
* **destination**: the node to which we want to find a path
* **explored**: a mapping of every node (key) to the move that it took to get to the node (value)

**returns** List[Tuple[int, int]]: a list of moves to take from destination to reach start.

In [9]:
def path(destination: Tuple[int, int], start: Tuple[int, int], explored: Dict[Tuple[int, int], Tuple[int, int]]) -> List[Tuple[int, int]]:
    curr = start
    stack = []

    while curr != destination:
        move = explored[curr][0:2]
        stack.append(move)
        curr = (curr[0] - move[0], curr[1] - move[1])
        
    stack.reverse()
    return stack

In [10]:
# assertions/unit tests
explored = {(0, 0): (0, 0, 0), (0, 1): (0, 1, 12), (0, 2): (0, 1, 11), (0, 3): (0, 1, 10), (1, 3): (1, 0, 9), (2, 3): (1, 0, 8), (3, 3): (1, 0, 7), (4, 3): (1, 0, 6), (5, 3): (1, 0, 5), (6, 3): (1, 0, 4), (6, 4): (0, 1, 3), (6, 5): (0, 1, 2), (6, 6): (0, 1, 0)}
pt = path((0, 0), (6, 6), explored)

assert pt == [(0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 1), (0, 1), (0, 1)]

pt = path((0, 0), (3, 3), explored)
assert pt == [(0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (1, 0)]

pt = path((0, 0), (0, 0), explored)
assert pt == []

<a id="successors"></a>
## successors

`successors` takes a current state, the world map, and a list of possible moves, and returns a list of children that can be reached from the current tile. The function computes possible successor states "on the fly" and uses the action set (`moves`) to determine the possible transitions available, rather than using an enumerated transition set. To determine valid transitions, the function checks the adjacent four tiles (up, right, down, left in that order) to see whether they are:
* traversable (not 🌋)
* in the world map boundaries

The function returns a list of successor states, with the first two elements corresponding to the child coordinates (x_to_check, y_to_check), and the third and fourth elements corresponding to the move taken to get to the child (move_x, move_y). **Used by:** [a_star_search](#a_star_search)

* **world**: a map of the world to check the conditions listed above
* **curr**: the current tile for which the successors will be returned
* **moves**: a list of moves to be checked

**returns** List[Tuple(x, y, move_x, move_y)]: a list of successor coordinate tuples


In [11]:
def successors( world: List[List[str]], curr: Tuple[int, int], moves: List[Tuple[int, int]]) -> List[Tuple[int, int, int, int]]:
    children = []
    for (move_x, move_y) in moves:
        x_to_check = curr[0] + move_x
        y_to_check = curr[1] + move_y
        if  y_to_check < len(world) and y_to_check >= 0 and x_to_check < len(world[y_to_check]) and x_to_check >= 0 and world[y_to_check][x_to_check] != "🌋":
            children.append((x_to_check, y_to_check, move_x, move_y))
    return children

In [12]:
# assertions/unit tests
curr = [0, 0]
actual_children = successors(small_world, curr, MOVES)
assert actual_children == [(1, 0, 1, 0), (0, 1, 0, 1)]

curr = [6, 6]
actual_children = successors(small_world, curr, MOVES)
assert actual_children == [(6, 5, 0, -1), (5, 6, -1, 0)]

# volcano is located at (4, 2) - verify volcano is not added to successors
curr = [3, 2]
actual_children = successors(full_world, curr, MOVES)
assert actual_children == [(3, 1, 0, -1), (3, 3, 0, 1), (2, 2, -1, 0)]


<a id="heuristic"></a>
## heuristic
`heuristic` takes a current position and a goal position and returns the estimated cost from the current position to the goal. To do so, it takes the Manhatten distance from the current to the goal position. This is an *admissible* heuristic, since it approximates the minimum cost path (it assumes the path is all tiles of weight 1, and we can only move in cartesian directions). **Used by:** [a_star_search](#a_star_search), [compute_cost](#compute_cost)

* **curr**: the current tile at which the heuristic is being approximated
* **goal**: the goal of the `a_star_search` algorithm

**returns** int: the estimated cost of reaching goal from curr


In [13]:
def heuristic(curr: Tuple[int, int], goal: Tuple[int, int]) -> int:
    return abs(goal[1] - curr[1]) + abs(goal[0] - curr[0])

In [14]:
# assertions/unit tests
curr = [0, 0]
goal = [6, 6]
actual_heuristic = heuristic(curr, goal)
assert actual_heuristic == 12

curr = [5, 5]
actual_heuristic = heuristic(curr, goal)
assert actual_heuristic == 2

curr = [6, 6]
actual_heuristic = heuristic(curr, goal)
assert actual_heuristic == 0

<a id="compute_cost"></a>
## compute_cost

`compute_cost` takes the current x-y position, the move taken to get there, the goal tile, the explored list, the world map, and a list of costs and computes the cost of reaching the goal from the current position. By looking up the position and the previous move, it looks up the total cost to the parent node from the explored list and adds the cost of traversing to the current tile, as well as the heuristic cost to reach the goal from the current tile. It returns a total cost value for `child`. **Uses**: [heuristic](#heuristic). **Used by:** [a_star_search](#a_star_search).

* **child**: the coordinates of the current child being computed.
* **move**: the move taken to reach the current child
* **goal**: the goal tile of the `a_star_search`
* **explored**: the dictionary of nodes previously explored, including the parent of `child`
* **world**: the world map
* **costs**: a dictionary of costs for each tile type in `world`

**returns** int: the total cost of the current path estimated by the cost to reach the current node and the heuristic cost.

In [15]:
def compute_cost(curr: Tuple[int, int], move: Tuple[int, int], goal: Tuple[int, int], explored: Dict[Tuple[int, int], Tuple[int, int, int]], world: List[List[str]], costs: Dict[str, int]) -> int:
    if curr == goal:
        return 0
    parent = (curr[0] - move[0], curr[1] - move[1])
    total_cost = explored[parent][2] if parent in explored else 0
    total_cost += costs[world[curr[1]][curr[0]]] + heuristic(curr, goal)
    return total_cost

In [16]:
# assertions/unit tests
explored = {(0, 0): (0, 0, 0), (1, 0): (1, 0, 12), (2, 0): (1, 0, 11), (3, 0): (1, 0, 10), (3, 1): (0, 1, 9), (3, 2): (0, 1, 8), (3, 3): (0, 1, 7), (3, 4): (0, 1, 6), (3, 5): (0, 1, 5), (3, 6): (0, 1, 4), (4, 6): (1, 0, 3), (5, 6): (1, 0, 2), (6, 6): (1, 0, 1)}
actual_cost = compute_cost((0, 1), (0, 1), (6, 6), explored, small_world, COSTS)
assert actual_cost == 12

actual_cost = compute_cost((1, 0), (1, 0), (6, 6), explored, small_world, COSTS)
assert actual_cost == 14

actual_cost = compute_cost((6, 6), (6, 5), (6, 6), explored, small_world, COSTS)
assert actual_cost == 0

<a id="pretty_print_helper"></a>
## pretty_print_helper

`pretty_print_helper` takes in a move and returns a corresponding character that represents that move on the world map. **Used by**: [pretty_print_path](#pretty_print_path)

* **move**: a tuple representing a move to be taken

**returns** str: an emoji string that represents the move

In [17]:
def pretty_print_helper(move: Tuple[int, int]) -> str:
    if move == (0, 1):
        return "⏬"
    elif move == (1, 0):
        return "⏩"
    elif move == (0, -1):
        return "⏫"
    else: 
        return "⏪"

In [18]:
# assertions/unit tests
move = (0, 1)
assert pretty_print_helper(move) == "⏬"

move = (1, 0)
assert pretty_print_helper(move) == "⏩"

move = (-1, 0)
assert pretty_print_helper(move) == "⏪"

<a id="a_star_search"></a>
## a_star_search

`a_star_search` is a state space search algorithm that utilizes a heuristic cost to determine which child path to visit at every step. This implementation takes a world map of emojis, a start coordinate, a goal coordinate, a list of costs for every emoji, a list of moves that represent the action set, and a heuristic function to determine the cost of a branch. 

The states in this algorithm are represented as key-value pairs: the keys are coordinate-tuples `(x, y)`, and the values are `(move_x, move_y, cost)` tuples that represent the move taken to get to the current state, as well as the cost required to reach the current state. The `frontier` is implemented as a priority stack of states, which is sorted by cost in descending order after pushing all children from each iteration using `priority_sort`. Similarly, the `explored` list is a dictionary of states.


**Uses:** [path](#path), [successors](#successors), [compute_cost](#compute_cost), [priority_sort](#priority_sort)
* **world** List[List[str]]: the actual context for the navigation problem.
* **start** Tuple[int, int]: the starting location of the bot, `(x, y)`.
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.
* **costs** Dict[str, int]: is a `Dict` of costs for each type of terrain in **world**.
* **moves** List[Tuple[int, int]]: the legal movement model expressed in offsets in **world**.
* **heuristic** Callable: is a heuristic function, $h(n)$.


**returns** List[Tuple[int, int]]: the offsets needed to get from start state to the goal as a `List`.


In [19]:
def a_star_search( world: List[List[str]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int], moves: List[Tuple[int, int]], heuristic: Callable) -> List[Tuple[int, int]]:
    frontier, explored = {start: (0, 0, 0)}, {}
    final_path = []
    while frontier:
        current_state = frontier.popitem()
        if current_state[0] == goal:
            explored[current_state[0]] = current_state[1]
            final_path = path(start, current_state[0], explored)
            break
        children = successors(world, current_state[0], moves)
        for (child_x, child_y, move_x, move_y) in children:
            if (child_x, child_y) not in explored:
                cost = compute_cost((child_x, child_y), (move_x, move_y), goal, explored, world, costs)
                if (child_x, child_y) not in frontier or frontier[(child_x, child_y)][2] > cost:
                    frontier[(child_x, child_y)] = (move_x, move_y, cost)
        frontier = priority_sort(frontier)
        explored[current_state[0]] = current_state[1]
    return final_path

<a id="pretty_print_path"></a>
## pretty_print_path

`pretty_print_path` takes in a world map, a path, a start and goal position, and a list of costs, and returns the cost of traversing the path and also prints the new world with an emoji-art path drawn from the start to the goal. The goal is notated with a "🎁" symbol, and each of the four cardinal directions is represented by its own emoji. This function is **destructive**, as it replaces the original `world` tiles with direction tiles. **Uses:** [pretty_print_helper](#pretty_print_helper).

* **world** List[List[str]]: the world (terrain map) for the path to be printed upon.
* **path** List[Tuple[int, int]]: the path from start to goal, in offsets.
* **start** Tuple[int, int]: the starting location for the path.
* **goal** Tuple[int, int]: the goal location for the path.
* **costs** Dict[str, int]: the costs for each action.

**returns** int - The path cost.


<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        Does your output of pretty_print_path really look like the specification? Go check again.
    </p>
</div>

In [20]:
def pretty_print_path( world: List[List[str]], path: List[Tuple[int, int]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int]) -> int:
    curr, path_cost = start, 0
    for move in path:
        path_cost += costs[world[curr[1]][curr[0]]]
        world[curr[1]][curr[0]] = pretty_print_helper(move)
        curr = (curr[0] + move[0], curr[1] + move[1])
    world[goal[1]][goal[0]] = "🎁"
        
    for row in world: 
        print("".join(row))
    return path_cost

## Problems

## Problem 1. 

Execute `a_star_search` and `pretty_print_path` for the `small_world`.

If you change any values while developing your code, make sure you change them back! (Better yet, don't do it. Copy them elsewhere and change the values, then delete those experiments).

In [21]:
small_start = (0, 0)
small_goal = (len(small_world[0]) - 1, len(small_world) - 1)
small_path = a_star_search(small_world, small_start, small_goal, COSTS, MOVES, heuristic)
small_path_cost = pretty_print_path(small_world, small_path, small_start, small_goal, COSTS)
print(f"total path cost: {small_path_cost}")
print(small_path)

⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏩⏩⏩⏩⏩⏩⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲🎁
total path cost: 12
[(0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 1), (0, 1), (0, 1)]


## Problem 2

Execute `a_star_search` and `print_path` for the `full_world`

In [22]:
full_start = (0, 0)
full_goal = (len(full_world[0]) - 1, len(full_world) - 1)
full_path = a_star_search(full_world, full_start, full_goal, COSTS, MOVES, heuristic)
full_path_cost = pretty_print_path(full_world, full_path, full_start, full_goal, COSTS)
print(f"total path cost: {full_path_cost}")
print(full_path)

⏬🌾🌾🌾🌾🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾
⏬🌾🌾🌾🌾🌾🌾🌲🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🌋🌋🌋🌋🌋🌋🌋🌾🌾
⏬🌾🌾🌾🌋🌋🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌋🌋🌋⛰⛰⛰🌋🌋⛰⛰
⏬🌾🌾🌾⛰🌋🌋🌋🌲🌲🌲🌲🐊🐊🌲🌲🌲🌲🌲🌾🌾⛰⛰🌋🌋⛰🌾
⏬🌾🌾⛰⛰🌋🌋🌲🌲🌾🌾🐊🐊🐊🐊🌲🌲🌲🌾🌾🌾⛰🌋🌋🌋⛰🌾
⏬⛰⛰⛰🌋🌋⛰⛰🌾🌾🌾🌾🐊🐊🐊🐊🐊🌾🌾🌾🌾🌾⛰🌋⛰🌾🌾
⏬⛰⛰🌋🌋⛰⛰🌾🌾🌾🌾⛰🌋🌋🌋🐊🐊🐊🌾🌾🌾🌾🌾⛰🌾🌾🌾
⏬🌾⛰⛰⛰⛰⛰🌾🌾🌾🌾🌾🌾⛰🌋🌋🌋🐊🐊🐊🌾🌾⛰⛰⛰🌾🌾
⏬🌾⏩⏩⏩⏩⏬🌾🌾🌾🌾🌾⛰⛰🌋🌋🌾🐊🐊🌾🌾⛰⛰⛰🌾🌾🌾
⏩⏩⏫🐊🐊🐊⏩⏬⛰⛰⛰🌋🌋🌋🌋🌾🌾🌾🐊🌾⛰⛰⛰🌾🌾🌾🌾
🌾🌾🐊🐊🐊🐊🐊⏬⛰⛰🌋🌋🌋⛰🌾🌾🌾🌾🌾⛰🌋🌋🌋⛰🌾🌾🌾
🌾🐊🐊🐊🐊🐊⏬⏪⛰🌋🌋⛰🌾🌾🌾🌾🐊🐊🌾🌾⛰🌋🌋⛰🌾🌾🌾
🐊🐊🐊🐊🐊🌾⏬⛰⛰🌋🌋⛰🌾🐊🐊🐊🐊🌾🌾🌾⛰🌋⛰🌾🌾🌾🌾
🌾🐊🐊🐊🐊🌾⏬⛰🌲🌲⛰🌾🌾🌾🌾🐊🐊🐊🐊🌾🌾⛰🌾🌾🌾🌾🌾
🌾🌾🌾🌾🌋🌾⏬🌲🌲🌲🌲⛰⛰⛰⛰🌾🐊🐊🐊🌾🌾⛰🌋⛰🌾🌾🌾
🌾🌾🌾🌋🌋🌋⏬🌲🌲🌲🌲🌲🌋🌋🌋⛰⛰🌾🐊🌾⛰🌋🌋⛰🌾🌾🌾
🌾🌾🌋🌋🌲🌲⏬🌲🌲🌲🌲🌲🌲🌲🌋🌋🌋🌾🌾🌋🌋🌋🌾🌾🌾🌾🌾
🌾🌾🌾🌋🌋🌲⏬🌲🌲🌲🌲🌲🌲🌲🌲🌲🌋🌋🌋🌋🌾🌾🌾🌾🌾🌾🌾
🌾🌾🌾🌋🌋🌋⏩⏬🌲🌲🌲🌲🌲🌲🌾🌾🌾⛰⛰🌾🌾🌾🌾🌾🌾🌾🌾
🌾🌾🌾🌾🌋🌋🌋⏩⏬🌲🌲🌲🌲⏩⏩⏬🌾🌾🌾🌾🌾🌾🌾🐊🐊🐊🐊
🌾🌾⛰⛰⛰⛰🌋🌋⏩⏩⏩⏩⏩⏫🌋⏬🌾🌾🌾🌾🐊🐊🐊🐊🐊🐊🐊
🌾🌾🌾🌾⛰⛰⛰🌋🌋🌋🌲🌲🌋🌋🌾⏩⏩⏩⏩⏬🐊🐊🐊🐊🐊🐊🐊
🌾🌾🌾🌾🌾🌾⛰⛰⛰🌋🌋🌋🌋🌾🌾🌾🌾⛰⛰⏩⏬🐊🐊🐊🐊🐊🐊
🌾⛰⛰🌾🌾⛰⛰⛰⛰⛰🌾🌾🌾🌾🌾⛰⛰🌋🌋⛰⏩⏬🐊🐊🐊🐊🐊
⛰🌋⛰⛰⛰⛰🌾🌾🌾🌾🌾🌋🌋🌋⛰⛰🌋🌋🌾🌋🌋⏩⏩⏬🐊🐊🐊
⛰🌋🌋🌋⛰🌾🌾🌾🌾🌾⛰⛰🌋🌋🌋🌋⛰⛰⛰⛰🌋🌋🌋⏬🐊🐊🐊
⛰⛰🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾⛰⛰⛰⛰⛰🌾🌾🌾🌾⛰⛰⏩⏩⏩🎁
total path cost: 122
[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (0, -1), (1, 0), (1, 0), (1, 0), (1, 0), (0, 1), (1, 0), (0, 1), (0, 1), (-1, 0), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 

## Comments

(This is the place to leave me comments)

## To think about for future assignments...

This first assignment may not have been difficult for you if you've encountered A* search before in your Algorithms course. In preparation for future assignments that build on State Space Search, you can think about the following or even do an implementation if you like. You should **not** submit it as part of this assignment.

In several future assignments, we will have a need for a "plain ol'" Depth First Search algorithm.

1. Implement DFS Search to solve the problem presented in this programming assignment. Try to be as general as possible (don't hard code anything you can pass as a formal parameter).
2. Can you implement DFS Search as a higher order function and supply your own `is_goal`, `successors`, and `path` functions? How do you handle *state*?
3. Can you write a version of DFS that returns all the solutions?

In one future assignment a Breadth First Search algorithm will be very handy. Can you implement a search algorithm that changes whether it uses DFS or BFS by parameterization?

## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you follow the Programming Requirements on Canvas?

Do not submit any other files.